## Web Scraping - Selenium

Code credit:
https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [3]:
#Installing required packages
!pip install selenium

In [4]:
!pip install Pillow

In [5]:
# Updating pip 
!pip install --upgrade pip

In [1]:
# Importing required packages
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import requests 
import io
from PIL import Image, ImageDraw
import hashlib
import os

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Give permission to load an app from an unknown developer
!xattr -d com.apple.quarantine chromedriver

xattr: chromedriver: No such xattr: com.apple.quarantine


In [3]:
def fetch_image_urls_util(url):
    images = []
    # Open main window with URL A
    with webdriver.Chrome(executable_path="./chromedriver") as wd:

        # Switch to the new window and open URL B
        try:
            wd.get(url)
        except:
            return []

        thumbnail_results = wd.find_elements_by_css_selector("img.n3VNCb")

        for img in thumbnail_results:
            if img.get_attribute('src') and 'http' in img.get_attribute('src'):
                images.append(img.get_attribute('src'))
                
    return images

In [27]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query + " face"))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[50:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(15)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
                print("Load more button clicked!")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [28]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [29]:
def search_and_download(search_term:str, service, target_path='./datasets', number_images=50):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(service=service) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        #res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1,target_path=target_path,search_term=search_term)
    try:    
        for elem in res:
            persist_image(target_folder,elem)
    except Exception as e:
        print(e)

In [33]:
query = ["Cristiano","Oprah Winfrey"]

s = Service('./chromedriver')

for q in query:
    search_and_download(q, s)

<ipython-input-27-57f15ffa19e9>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100
Found: 0 image links, looking for more ...
'NoneType' object is not iterable
Found: 100 search results. Extracting links from 0:100
Found: 0 image links, looking for more ...
'NoneType' object is not iterable


In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, service, wd, sleep_between_interactions:int=1, target_path = None, search_term = None):
    
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    image_count2 = 0
    results_start = 0
    d = {}
    
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception as e:
                print(e)
                continue
            
            links = wd.find_elements_by_css_selector("a[jsname='sTFXNd']")
#            print("Found %d links" % len(links))
            
            for link in links:
                if link.get_attribute('href') and 'http' in link.get_attribute('href'):
#                    print("Found valid link")
                    if link.get_attribute('href') not in d:
#                        print("Found new link")
                        d[link.get_attribute('href')] = True
                        getactualurl = fetch_image_urls_util(link.get_attribute('href'))
#                    else:
#                        print("Link already added.")
                    for imageurl in getactualurl:
                        if imageurl is not None:
#                            print(imageurl)
                            image_urls.add(imageurl)
#                else:
#                    print("Invalid link.")
            
            image_count2 = len(image_urls)
            
            if image_count2 >= max_links_to_fetch/10:
                print(f"Found: {len(image_urls)} image links, saving!")
                try:    
                    for elem in image_urls:
                        persist_image(target_folder,elem)
                except Exception as e:
                    print(e)
                image_urls = set()
                d = {}

            image_count += image_count2
                
        #image_count = len(image_urls)

        if len(image_urls) >= max_links_to_fetch:
            print(f"Found: {len(image_urls)} image links, done!")
            break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = image_count

    print(image_urls)
    return image_urls